# Social Network Analysis - Amber Heard Case - Twitter

## Preparing Data Set - P2

In [1]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
import warnings
import re
import seaborn as sb
import re

warnings.filterwarnings("ignore")

import plotly.io as pio
from helpers import *

pio.renderers
# pio.renderers.default = "svg"
svg_renderer = pio.renderers["svg"]
svg_renderer.width = 950
svg_renderer.height = 550

%matplotlib inline
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import ast
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Leap\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
data_path = "../../twitter_data/"

In [3]:
data_2022 = [_ for _ in os.listdir(data_path) if "2022" in _]

In [4]:
data_2022

['2017-2022 Twitter all_quotes retweets Amber Heard Timeline_Data.csv',
 '2022 Tweets_Metrics.csv',
 '2022 Twitter Accounts Jan-April_Amber Heard Data.csv',
 '2022 Twitter daily_creation Jan-April_Amber Heard Data.csv',
 '2022 Twitter Tweets Jan-April_Amber Heard Data.csv',
 '2022 Twitter Tweets_lite Jan-April_Amber Heard Data.csv',
 '2022 Twitter users_created_in_2022_Jan-April_Amber Heard Data.csv',
 '2022_prepared_tweets.csv',
 'Twitter 2017-2022 all_comments on Amber Heard Profile.csv',
 'Twitter 2017-2022 all_comments on Amber Heard Profile.json',
 'Twitter comments_2022 on Profile Amber Heard.csv']

## Load Twitter Prepared Data

In [5]:
df_tweets_2022_prepared = pd.read_csv(data_path+"2022_prepared_tweets.csv")

In [11]:
df_tweets_2022_prepared["created_at"] = pd.to_datetime(df_tweets_2022_prepared["created_at"])
df_tweets_2022_prepared["user.created_at"] = pd.to_datetime(df_tweets_2022_prepared["user.created_at"])

In [12]:
df_tweets_2022_prepared.drop(columns=["user.description"], inplace=True)

In [14]:
df_tweets_2022_prepared.rename(
    columns={
        "id_str": "id",
        "user.id_str": "user_id",
        "user.screen_name": "user_name",
        "user.created_at": "user_created_at",
        "tokens": "text_tokens",
    },
    inplace=True,
)

In [15]:
data_investigation(df_tweets_2022_prepared)

Data Head


,id,created_at,user_id,user_name,user_created_at,mentions,n_mentions,hashtags,n_hashtags,clean_text,text_tokens,user_description
0,1477504983130116096,2022-01-02 05:00:13+00:00,1.149795e+08,seano999,2010-02-17 06:20:14+00:00,"['la77465262', 'Melinda15199317', 'Dior']",3,['JohnnyDepp'],1,so why is only bringing a civil case against ...,"['bringing', 'civil', 'case', 'amber', 'heard'...","young guy, varied interests. socialist. europe..."
1,1477503751716065280,2022-01-02 04:55:19+00:00,1.426227e+18,WonderWilson18,2021-08-13 17:00:36+00:00,[],0,[],0,only thing i want them to leave alone is the f...,"['thing', 'leave', 'alone', 'first', 'wonder',...","so fuck your rules man your favorite k2, avril..."


______________________________________________________________________________

Data Shape


'The data has 573780 rows and 12 columns'

______________________________________________________________________________

Columns


...

______________________________________________________________________________

Percentge of Nulls in Each Column


user_description    0.207539
clean_text          0.101318
text_tokens         0.101318
user_id             0.089804
user_name           0.089804
user_created_at     0.089804
created_at          0.000003
id                  0.000000
mentions            0.000000
n_mentions          0.000000
hashtags            0.000000
n_hashtags          0.000000
dtype: float64

______________________________________________________________________________

Numeric Columns' Staticts


,user_id,n_mentions,n_hashtags
count,5.222520e+05,573780.000000,573780.000000
mean,7.100235e+17,0.339529,1.035345
std,6.337171e+17,0.964897,1.588022
min,1.968000e+03,0.000000,0.000000
25%,4.470993e+08,0.000000,0.000000
50%,9.302652e+17,0.000000,0.000000
75%,1.325802e+18,0.000000,1.000000
max,1.519822e+18,50.000000,23.000000


______________________________________________________________________________



**10 % of text (tweets) contains only emojis**

## User Desription into Tokens

In [17]:
all_stop_words = []
for lang in stopwords.fileids():
    all_stop_words.append(stopwords.words(lang))
all_stop_words = [item for sublist in all_stop_words for item in sublist]

In [18]:
def tokenize(text: str) -> list:
    if str(text) == "nan":
        return None
    else:
        return [i for i in [i for i in word_tokenize(text) if i not in all_stop_words] if i.isalnum()]    

In [20]:
df_tweets_2022_prepared["user_description"] = np.where(
    df_tweets_2022_prepared.user_description == "",
    np.nan,
    df_tweets_2022_prepared.user_description,
)

In [21]:
import time
start_time = time.time()
df_tweets_2022_prepared["user_description_tokens"] = df_tweets_2022_prepared.user_description.apply(lambda x: tokenize(str(x)))
print("--- %s seconds ---" % (time.time() - start_time))

--- 1071.531247138977 seconds ---


In [22]:
df_tweets_2022_prepared

,id,created_at,user_id,user_name,user_created_at,mentions,n_mentions,hashtags,n_hashtags,clean_text,text_tokens,user_description,user_description_tokens
0,1477504983130116096,2022-01-02 05:00:13+00:00,1.149795e+08,seano999,2010-02-17 06:20:14+00:00,"['la77465262', 'Melinda15199317', 'Dior']",3,['JohnnyDepp'],1,so why is only bringing a civil case against ...,"['bringing', 'civil', 'case', 'amber', 'heard'...","young guy, varied interests. socialist. europe...","[young, guy, varied, interests, socialist, eur..."
1,1477503751716065280,2022-01-02 04:55:19+00:00,1.426227e+18,WonderWilson18,2021-08-13 17:00:36+00:00,[],0,[],0,only thing i want them to leave alone is the f...,"['thing', 'leave', 'alone', 'first', 'wonder',...","so fuck your rules man your favorite k2, avril...","[fuck, rules, favorite, k2, avril, lavigne, wo..."
2,1477499814719983617,2022-01-02 04:39:41+00:00,1.427359e+18,StevenJCurry,2021-08-16 19:58:03+00:00,"['soldierboy43001', 'ZakReckless']",2,[],0,yeah but snyder isn't involved in the flash so...,"['yeah', 'snyder', 'involved', 'flash', 'would...",NaN,None
3,1477497740498292738,2022-01-02 04:31:26+00:00,1.354477e+18,Vincent72516869,2021-01-27 17:11:08+00:00,"['IIMissMax', 'bg98021', 'GerberKawasaki']",3,[],0,you forgot he hook up with amber heard . i am ...,"['forgot', 'hook', 'amber', 'heard', 'sure', '...",NaN,None
4,1477496477484212225,2022-01-02 04:26:25+00:00,1.076943e+18,YoonJaysSimp,2018-12-23 20:50:49+00:00,[],0,[],0,i never seen aquaman nor will i because amber ...,"['never', 'seen', 'aquaman', 'amber', 'heard']",bl content * anime * manga * manhwa * nsfw * s...,"[bl, content, anime, manga, manhwa, nsfw, simm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573775,1515569901158600704,2022-04-17 05:56:37+00:00,2.701354e+07,GhelSandra,2009-03-27 14:25:52+00:00,[],0,[],0,gender card pulled out. they are now spinning ...,"['gender', 'card', 'pulled', 'spinning', 'make...",tomasino est 2000something art educator.person...,"[tomasino, 2000something, art, sayaka, akimoto..."
573776,1515569890391961601,2022-04-17 05:56:34+00:00,1.418420e+18,lilithrising_,2021-07-23 03:59:18+00:00,[],0,[],0,amber heard is a libra moon. ive known a few l...,"['amber', 'heard', 'libra', 'moon', 'ive', 'kn...",NaN,None
573777,1515569612686970883,2022-04-17 05:55:28+00:00,3.303069e+09,CJMoss007,2015-08-01 07:03:17+00:00,[],0,"['JusticeForJohnnyDepp', 'JusticeForJohnnyDepp...",2,NaN,NaN,#justiceforjohnnydepp,[justiceforjohnnydepp]
573778,1515569604818575368,2022-04-17 05:55:26+00:00,1.319471e+18,Jankem69420,2020-10-23 02:50:36+00:00,[],0,[],0,"amber heard is dating eve fartlow, chelsea man...","['amber', 'heard', 'dating', 'eve', 'fartlow',...",''virtue-signalling geopolitically ignorant ca...,"[geopolitically, ignorant, cartoon, mouse, lam..."


In [23]:
df_tweets_2022_prepared.columns

Index(['id', 'created_at', 'user_id', 'user_name', 'user_created_at',
       'mentions', 'n_mentions', 'hashtags', 'n_hashtags', 'clean_text',
       'text_tokens', 'user_description', 'user_description_tokens'],
      dtype='object')

In [26]:
df_tweets_2022_prepared = df_tweets_2022_prepared[
    [
        "id",
        "created_at",
        "clean_text",
        "text_tokens",
        "mentions",
        "n_mentions",
        "hashtags",
        "n_hashtags",
        "user_id",
        "user_name",
        "user_created_at",
        "user_description",
        "user_description_tokens",
    ]
]

In [27]:
def remove_stopwords(tokens):
    if tokens is None:
        return None
    else:
        return [i for i in tokens if i not in all_stop_words]

In [28]:
df_tweets_2022_prepared["user_description_tokens"] = df_tweets_2022_prepared[
    "user_description_tokens"
].apply(lambda x: remove_stopwords(x))

In [34]:
df_tweets_2022_prepared

,id,created_at,clean_text,text_tokens,mentions,n_mentions,hashtags,n_hashtags,user_id,user_name,user_created_at,user_description,user_description_tokens
0,1477504983130116096,2022-01-02 05:00:13+00:00,so why is only bringing a civil case against ...,"['bringing', 'civil', 'case', 'amber', 'heard'...","['la77465262', 'Melinda15199317', 'Dior']",3,['JohnnyDepp'],1,1.149795e+08,seano999,2010-02-17 06:20:14+00:00,"young guy, varied interests. socialist. europe...","[young, guy, varied, interests, socialist, eur..."
1,1477503751716065280,2022-01-02 04:55:19+00:00,only thing i want them to leave alone is the f...,"['thing', 'leave', 'alone', 'first', 'wonder',...",[],0,[],0,1.426227e+18,WonderWilson18,2021-08-13 17:00:36+00:00,"so fuck your rules man your favorite k2, avril...","[fuck, rules, favorite, k2, avril, lavigne, wo..."
2,1477499814719983617,2022-01-02 04:39:41+00:00,yeah but snyder isn't involved in the flash so...,"['yeah', 'snyder', 'involved', 'flash', 'would...","['soldierboy43001', 'ZakReckless']",2,[],0,1.427359e+18,StevenJCurry,2021-08-16 19:58:03+00:00,NaN,None
3,1477497740498292738,2022-01-02 04:31:26+00:00,you forgot he hook up with amber heard . i am ...,"['forgot', 'hook', 'amber', 'heard', 'sure', '...","['IIMissMax', 'bg98021', 'GerberKawasaki']",3,[],0,1.354477e+18,Vincent72516869,2021-01-27 17:11:08+00:00,NaN,None
4,1477496477484212225,2022-01-02 04:26:25+00:00,i never seen aquaman nor will i because amber ...,"['never', 'seen', 'aquaman', 'amber', 'heard']",[],0,[],0,1.076943e+18,YoonJaysSimp,2018-12-23 20:50:49+00:00,bl content * anime * manga * manhwa * nsfw * s...,"[bl, content, anime, manga, manhwa, nsfw, simm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573775,1515569901158600704,2022-04-17 05:56:37+00:00,gender card pulled out. they are now spinning ...,"['gender', 'card', 'pulled', 'spinning', 'make...",[],0,[],0,2.701354e+07,GhelSandra,2009-03-27 14:25:52+00:00,tomasino est 2000something art educator.person...,"[tomasino, 2000something, art, sayaka, akimoto..."
573776,1515569890391961601,2022-04-17 05:56:34+00:00,amber heard is a libra moon. ive known a few l...,"['amber', 'heard', 'libra', 'moon', 'ive', 'kn...",[],0,[],0,1.418420e+18,lilithrising_,2021-07-23 03:59:18+00:00,NaN,None
573777,1515569612686970883,2022-04-17 05:55:28+00:00,NaN,NaN,[],0,"['JusticeForJohnnyDepp', 'JusticeForJohnnyDepp...",2,3.303069e+09,CJMoss007,2015-08-01 07:03:17+00:00,#justiceforjohnnydepp,[justiceforjohnnydepp]
573778,1515569604818575368,2022-04-17 05:55:26+00:00,"amber heard is dating eve fartlow, chelsea man...","['amber', 'heard', 'dating', 'eve', 'fartlow',...",[],0,[],0,1.319471e+18,Jankem69420,2020-10-23 02:50:36+00:00,''virtue-signalling geopolitically ignorant ca...,"[geopolitically, ignorant, cartoon, mouse, lam..."


In [35]:
df_tweets_2022_prepared.to_csv(data_path+"2022_prepared_tweets.csv", index=False)